In [ ]:
# {
#   "by": "css",                     // "xpath", "css", "id", "name", etc.
#   "value": "your-selector-here",  // Selector string
#   "wait": 0.5,                    // (Optional) seconds to wait before doing anything
#   "wait_until": "visible",        // "clickable", "visible", "present", etc.
#   "timeout": 10,                  // Max time to wait for `wait_until` condition

#   "action": "click",              // "click", "scrape", "check", "screenshot", etc.
#   "log_message": "Describe this action clearly",  // Log for debugging

#   "new_window": false,            // (Optional) If this click opens new tab
#   "return_to_base": false,        // (Optional) If true, go back to base window after action

#   "multiple": false,              // (Only for scrape) True to loop over many matches

#   "scrape_fields": {              // (Only for scrape) Define keys and child-selectors
#     "title": "h3 a",
#     "price": "p.price_color"
#   },

#   "attribute": null,              // (Only for scrape) If scraping attribute instead of text

#   "check_text": null,             // (Optional) For action: "check" — verify expected text
#   "screenshot_path": null         // (Optional) For action: "screenshot" — save path
# }


In [6]:
import time, base64, os, re,json5, pprint
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin

In [ ]:
import json5
from dateutil.parser import parse
import re

path = r'example.json5'

def extract_date(text: str) -> str:
    if not isinstance(text, str):
        print("Invalid data type. Expected string.")
        return ""
    
    output_format = "%Y%m%d"
    date_pattern = r"(\d{2}[.\-/]+\d{2}[.\-/]+\d{4}|\d{1,2}\s*(?:th|st|rd|nd)\s*[A-Za-z]+\s*\d{4}|\d{2}[\.\-\/]+[A-Za-z]+[\.\-\/]+\d{4}|\d{2}\s*[A-Za-z]+\s*\d{4})"
    matches = re.findall(date_pattern, text, re.IGNORECASE)
    
    if matches:
        date_str = " ".join(matches)
        dt_object = parse(date_str, fuzzy=True)
        return dt_object.strftime(output_format)
    
    # print("Date not found.")
    return text

with open(path, "r+", encoding="utf-8") as file:
    data = json5.load(file)

records = data.get("records", [])

for record in records:
    
    print(record["bank_name"])
    response_data = record.get("response", {})
    if not response_data:
        continue

    extracted_dates = {}

    for block in response_data.values():
        scrape_data = block.get("scrape_data", {})
        if not scrape_data:
            continue

        cleaned_data = {key: extract_date(value) for key, value in scrape_data.items()}
        extracted_dates.update(cleaned_data)

    print(extracted_dates)
    record["response"] = extracted_dates

with open("up20250801.json5", "r+", encoding="utf-8") as file:
   json5.dump(data,file)

In [32]:
data = '\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t<tr>\n'

# data = "Column1\t\t\tColumn2\t\tColumn3"

cleaned = re.sub(r'\t+', ' ', data)

print(cleaned)
print(data)

 <tr>

																<tr>



In [45]:
#BANK OF BARODA
options = Options()
# options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

path = "https://www.bankofbaroda.in/interest-rate-and-service-charges/deposits-interest-rates"
driver = webdriver.Chrome(options=options)
driver.get(path)
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "component-content"))
)

# headers = driver.find_elements(By.CSS_SELECTOR,".field-heading")

tables = driver.find_elements(By.CSS_SELECTOR,".table-resopnsive")

# for header in headers:
#     print(header.text)

for idx,content in enumerate(tables):
    
    tables = content.find_elements(By.TAG_NAME,"table")
    for table in tables:
        data = table.get_attribute("outerHTML")
        data = re.sub(r'\t+', '', data)
        with open(f"example_{idx}.html","w+", encoding="utf8") as file:
            file.write(data)
    # pprint.pprint(data)
# driver.quit()


#other data
# path = 'https://www.bankofbaroda.in/interest-rate-and-service-charges/retail-loans-interest-rates'
# driver.get(path)

# WebDriverWait(driver, 5).until(
#     EC.presence_of_element_located((By.CLASS_NAME, "col-sm-12"))
# )

# dater = driver.find_element(By.CSS_SELECTOR,"h4+ p")
# print(dater.text)

time.sleep(10)
driver.quit()

In [ ]:
df = pd.read_html("example_4.html")

In [56]:
df[0]

,Unnamed: 0,â¹ 3.00 Crores up to â¹ 5.00 Crores,â¹ 3.00 Crores up to â¹ 5.00 Crores.1,â¹ 3.00 Crores up to â¹ 5.00 Crores.2,â¹ Above 5.00 Crores up to â¹ 10.00 Crores,â¹ Above 5.00 Crores up to â¹ 10.00 Crores.1,â¹ Above 5.00 Crores up to â¹ 10.00 Crores.2,â¹ Above 5.00 Crores up to â¹ 10.00 Crores.3,â¹ Above 5.00 Crores up to â¹ 10.00 Crores.4
0,TENOR,Rs. 3.00 Crs to upto Rs. 4.00 Crs,Above Rs. 4.00 Crs to upto Rs. 5.00 Crs,Above Rs. 5.00 Crs to upto Rs. 6.00 Crs,Above Rs. 6.00 Crs to upto Rs. 7.00 Crs,Above Rs. 7.00 Crs to upto Rs. 8.00 Crs,Above Rs. 8.00 Crs to upto Rs. 9.00 Crs,Above Rs. 9.00 Crs to upto Rs. 10.00 Crs,NaN
1,1 Year,6.25,6.25,6.25,6.25,6.25,6.25,6.25,NaN
2,Above 1Y to 15M,6.00,6.00,6.00,6.00,6.00,6.00,6.00,NaN
3,Above 15M to 2Y,6.00,6.00,6.00,6.00,6.00,6.00,6.00,NaN
4,Above 2Y to 3Y,5.50,5.50,5.50,5.50,5.50,5.50,5.50,NaN
5,Above 3Y to 5Y,5.00,5.00,5.00,5.00,5.00,5.00,5.00,NaN
6,Above 5Y and upto 10Y,4.50,4.50,4.50,4.50,4.50,4.50,4.50,NaN


In [22]:
#BANK OF INDIA
options = Options()
# options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")
path = 'https://bankofindia.co.in/interest-rate/saving-bank-deposit-rate'

driver = webdriver.Chrome(options=options)
driver.get(path)

WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "card-body"))
)

elements = driver.find_elements(By.CSS_SELECTOR,".th-exchange+ .th-exchange")

for elem in elements:
    print(f">> {elem.text}")

>> REVISED RATE OF INTEREST (% PER ANNUM)
W.E.F. 07.07.2025


In [25]:
options = Options()
# options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

path = 'https://canarabank.com/pages/deposit-interest-rates'
driver = webdriver.Chrome(options=options)
driver.get(path)

WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "main-content"))
)

savings = driver.find_elements(By.CSS_SELECTOR,".text-sec > p:nth-child(1) strong")

bulk_callable = driver.find_elements(By.CSS_SELECTOR,"p:nth-child(8) b")

bulk_non_callable = driver.find_elements(By.CSS_SELECTOR,"p:nth-child(11) b")

fncr_deposits = driver.find_elements(By.CSS_SELECTOR,".MsoNormal span")

for elem in savings:
    print(f">> {elem.text}")

for elem in bulk_callable:
    print(f">> {elem.text}")
    
for elem in bulk_non_callable:
    print(f">> {elem.text}")
    
for elem in fncr_deposits:
    print(f">> {elem.text}")
    
driver.quit()

>> DOMESTIC/NRO/NRE SAVINGS BANK DEPOSITS - W.E.F. 01.07.2025
>> The revised Rates of Interest for Bulk Deposits (Callable) w.e.f 10.07.2025 are as under:
>> The revised Rates of Interest for Bulk Deposits (Non-Callable) w.e.f 10.07.2025 are as under:
>>  
>>  
>> Interest rates on FCNR (B) Deposits w.e.f. 16.05.2025 (Interest rate % per annum)
>> 


In [4]:
import time
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options



#setup
driver = webdriver.Chrome() #controls the entire session
driver.get("https://ucobank.com/interest-rates-on-deposit-schemes")

#wait
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "mainbody"))
)

#toggle/click
toggles = driver.find_elements(By.CLASS_NAME, "collapsed")
for toggle in toggles:
    driver.execute_script("arguments[0].click();", toggle)  # safer than .click()
    time.sleep(0.5)

#tables
tables = driver.find_elements(By.CLASS_NAME, "table-responsive")

all_data = []
for table in tables:
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text.strip() for cell in cells]
        if row_data:
            all_data.append(row_data)

driver.quit()

#save
df = pd.DataFrame(all_data)
df.to_excel("interest_rates.xlsx", index=False, header=False)


In [3]:
import time, base64, os, re
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin


options = Options()
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

path = "https://www.indianbank.in/departments/deposit-rates/#!"

driver = webdriver.Chrome()
driver.get(path)

WebDriverWait(driver,5).until(
    EC.presence_of_element_located((By.CLASS_NAME,"container"))
)

rates = driver.find_element(By.CSS_SELECTOR, "tr+ .specificBackground td:nth-child(2) > strong")

# for rate in rates:
#     print("Start",rate.get_attribute("outerHTML"),"End")
print(rates.text)
driver.quit()

Revised w.e.f 09.06.2025
